## Kinematic transformation operator for lattice models

In [1]:
import numpy as np
%matplotlib widget
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt

In [2]:
%%html
<style>
div.jupyter-widgets.widget-label {display: none;}
</style>

In [3]:
u = np.array([[1,2,3],
              [2,3,4]], dtype=np.float_)
v = np.array([[3,2,5],
              [5,3,2]], dtype=np.float_)

In [4]:
np.einsum('...a,...a->...', u, v)

array([22., 27.])

In [5]:
A = np.einsum('...i,...j->...ij', u, v)

In [6]:
np.einsum('...ij,...j->...i', A, v)

array([[ 38.,  76., 114.],
       [ 76., 114., 152.]])

In [7]:
C = np.einsum('...ij,...jk->...ij', A, A)

In [8]:
np.einsum('ijk,...j,...k->...i', EPS, u, v)

NameError: name 'EPS' is not defined

### Kronecker delta

In [9]:
DELTA = np.identity(3)

### Levi Civita symbol

In [10]:
EPS = np.zeros((3, 3, 3), dtype='f')
EPS[(0, 1, 2), (1, 2, 0), (2, 0, 1)] = 1
EPS[(2, 1, 0), (1, 0, 2), (0, 2, 1)] = -1

### Index letters

To orientation in the formulas and in the code, the following 
index convention is used to in the sequel:

 * $I$ is a global node index, $I \in (0, n_I-1)$
 * $a,b,c,d$ are indexes of spatial directions $a,b,c,d \in (0,1,2)$
 * $L$ is a global index of a line, $L \in (0, n_L-1)$
 * $i$ is an index of a line node $i \in (0,1)$ 
 * $p$ is an index of kinematic category $p \in (0,1)$ where index $p = 0$ denotes translation and $p = 1$ denotes rotation

### Input arrays
The array of nodal coordinates in 3D and their links $L$: 

In [11]:
X_Ia = np.array(
    [
        [0, 0, 0],
        [0, 1, 0],
        [-1, 1, 0],
        [1, 1, 0]
    ], dtype=np.float_
)

I_Li = np.array(
    [
        [0, 2],
        [0, 3],
        [1, 2],
        [1, 3]
    ], dtype=np.int_
)
# X_Ia = np.array([[0, 0, 0], [1, 0, 0]], dtype=np.float_ )
# I_Li = np.array([[0,1]], dtype=np.int_)

```python
X_Ia = np.array([[0, 0, 0], [1, 0, 0]], dtype=np.float_ )
I_Li = np.array([[0,1]], dtype=np.int_)
```

### Rearange the nodes into an array with line and local node index

In [12]:
X_Lia = X_Ia[I_Li]
X_Lia

array([[[ 0.,  0.,  0.],
        [-1.,  1.,  0.]],

       [[ 0.,  0.,  0.],
        [ 1.,  1.,  0.]],

       [[ 0.,  1.,  0.],
        [-1.,  1.,  0.]],

       [[ 0.,  1.,  0.],
        [ 1.,  1.,  0.]]])

```python
X_Lia = X_Ia[I_Li]
```

Matplotlibs repeats the plot along the last dimension. Thus, rearange the array representing the links with nodal coordinates such that $L$ is the last dimension and the spatial coordinate is the first dimension. 

In [13]:
X_aiL = np.einsum('Lia->aiL', X_Lia)

Then, the lattice can be plotted in using the line.

In [14]:
_, ax = plt.subplots(1,1)
ax.plot(*X_aiL,color='black', marker="o");
ax.set_xlabel('x');
ax.set_ylabel('y');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
X_Lai = np.einsum('Lia->Lai', X_Lia)
fig = plt.figure()
ax = plt.axes(projection="3d")
for X_ai in X_Lai:
    x_line, y_line, z_line = X_ai
    ax.plot3D(x_line, y_line, z_line, 'gray');
x_points, y_points, z_points = np.einsum('Ia->aI', X_Ia)
ax.scatter3D(x_points, y_points, z_points, color='red');
ax.set_xlabel('X');
ax.set_ylabel('Y');
ax.set_zlabel('Z');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Construction of orthogonal basis
To provide a robust scheme for the identification of two tangential
vectors the concept presented here has been applied

https://orbit.dtu.dk/files/126824972/onb_frisvad_jgt2012_v2.pdf

In [16]:
# line vector v0
Tu_La = X_Lia[..., 1, :] - X_Lia[..., 0, :]
# Exclude the cases that a vector is perfectly aligned with an axis
I = np.fabs(Tu_La[..., 0]) > np.fabs(Tu_La[..., 2])
# a reference vector orthogonal to y-z projection - within the y-z plane 
Tvv_La = np.c_[0 * Tu_La[..., 0], -Tu_La[..., 2], Tu_La[..., 1]]
# a reference vector orthogonal to x-y projection - within the x-y plane
Tvv_La[I, :] = np.c_[-Tu_La[I, 1], Tu_La[I, 0], 0 * Tu_La[I, 0]]
# vector v1 - orthogonal to the line vector v0 and a reference vector
Tw_La = np.einsum('abc,...a,...b->...c', EPS, Tu_La, Tvv_La)
# vector v2 - orthogonal to v0 and v1
Tv_La = np.einsum('abc,...a,...b->...c', EPS, Tw_La, Tu_La)
# orthonormal bases
T_Lba = np.einsum('...bLa->...Lba', np.array([Tu_La, Tv_La, Tw_La]))

### Normalize to get orthonormal basis

In [17]:
norm_T_lb = 1. / np.sqrt(np.einsum(
    '...lba,...lba->...lb', T_Lba, T_Lba)
)
nT_Lba = np.einsum('...lb,...lba->...lba', norm_T_lb, T_Lba)
nT_Lba

array([[[-0.70710678,  0.70710678,  0.        ],
        [-0.70710678, -0.70710678,  0.        ],
        [ 0.        ,  0.        ,  1.        ]],

       [[ 0.70710678,  0.70710678,  0.        ],
        [-0.70710678,  0.70710678,  0.        ],
        [ 0.        ,  0.        ,  1.        ]],

       [[-1.        ,  0.        ,  0.        ],
        [ 0.        , -1.        ,  0.        ],
        [ 0.        ,  0.        ,  1.        ]],

       [[ 1.        ,  0.        ,  0.        ],
        [ 0.        ,  1.        ,  0.        ],
        [ 0.        ,  0.        ,  1.        ]]])

## Coordinates of interfaces between two linked aggregates
\begin{align}
    x^{\mathrm{m}}_{La} = \frac{1}{2} \delta_{ii} x_{Lia}
\end{align}

\begin{align}
    \Delta x^{m}_{Lia} = x^{\mathrm{m}}_{La} - x_{Lia}
\end{align}

In [18]:
DELTA2 = np.identity(2)
Xm_La = np.einsum('ii,Lia->La', DELTA2, X_Lia) / 2
Xm_Lia = Xm_La[..., np.newaxis, :]
dXm_Lia = Xm_Lia - X_Lia
dXm_Lia

array([[[-0.5,  0.5,  0. ],
        [ 0.5, -0.5,  0. ]],

       [[ 0.5,  0.5,  0. ],
        [-0.5, -0.5,  0. ]],

       [[-0.5,  0. ,  0. ],
        [ 0.5,  0. ,  0. ]],

       [[ 0.5,  0. ,  0. ],
        [-0.5,  0. ,  0. ]]])

```python
DELTA2 = np.identity(2)
Xm_La = np.einsum('ii,Lia->La', DELTA2, X_Lia) / 2
Xm_Lia = Xm_La[..., np.newaxis, :]
dXm_Lia = Xm_Lia - X_Lia
```



## Displacement of nodes $U_{Ia}$ and rotation of aggregates $\Phi_{Ia}$

In [19]:
U_Ia = np.array(
    [
        [0.4, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]
    ], dtype=np.float_
)

Phi_Ia = np.array(
    [
        [0, 0, 0.1 * np.pi],
        [0, 0, -0.1 * np.pi],
        [0, 0, 0.1 * np.pi],
        [0.3 * np.pi, 0, 0.1 * np.pi]
    ], dtype=np.float_
)
# U_Ia = np.array(
#     [[0, 0, 0],
#      [0, -0.1, 0]], dtype=np.float_
# )
# Phi_Ia = np.array(
#     [[0, 0, 0],
#     [0,0, np.pi/10]], dtype=np.float_
# )

In [20]:
U_Lia = U_Ia[I_Li]
U_Lia

array([[[0.4, 0. , 0. ],
        [0. , 0. , 0. ]],

       [[0.4, 0. , 0. ],
        [0. , 0. , 0. ]],

       [[0. , 0. , 0. ],
        [0. , 0. , 0. ]],

       [[0. , 0. , 0. ],
        [0. , 0. , 0. ]]])

## Global displacement of interface points

### Translation of aggregates mapped to interfaces
Distribute the translation of a center of grain gravity $u_{Ia}$ to the interface $i$ of the link $L$
\begin{align}
U^{\mathrm{m,trans}}_{Lia} = U_{I_{[Li]}a}
\end{align}

In [21]:
Um_trans_Lia = U_Ia[I_Li]

```python
Um_trans_Lia = U_Ia[I_Li]
```

### Rotations of aggregates
\begin{align}
    \Theta_{Lib} &= \Theta_{I[Li]a} \\
    U^{\mathrm{m,rot}}_{Lia} &=  \epsilon_{abc}  \Theta_{Lib} \Delta x^{\mathrm{m}}_{Lic}  
\end{align}

In [22]:
Phi_Lia = Phi_Ia[I_Li]
Um_rot_Lia = np.einsum('abc,...b,...c->...a',
                   EPS, Phi_Lia, dXm_Lia)

```python
Phi_Lia = Phi_Ia[I_Li]
Um_rot_Lia = np.einsum('abc,...b,...c->...a',
                   EPS, Phi_Lia, dXm_Lia)
```

The  global displacement at the interface is 
\begin{align}
    U^{\mathrm{m}}_{Lia} = U^{\mathrm{m,trans}}_{Lia} + U^{\mathrm{m,rot}}_{Lia}
\end{align}

In [23]:
Um_Lia = Um_trans_Lia + Um_rot_Lia
Um_Lia

array([[[ 0.24292037, -0.15707963,  0.        ],
        [ 0.15707963,  0.15707963,  0.        ]],

       [[ 0.24292037,  0.15707963,  0.        ],
        [ 0.15707963, -0.15707963, -0.4712389 ]],

       [[ 0.        ,  0.15707963,  0.        ],
        [ 0.        ,  0.15707963,  0.        ]],

       [[ 0.        , -0.15707963,  0.        ],
        [ 0.        , -0.15707963,  0.        ]]])

```python
Um_Lia = Um_trans_Lia + Um_rot_Lia
```

### Plot the displacements

In [24]:
Um_trans_aiL = np.einsum('Lia->aiL',Um_trans_Lia)
XU_aiL = X_aiL + Um_trans_aiL
XUm_aiL = np.einsum('Lia->aiL',(Xm_Lia + Um_Lia))
XUIm_aiL = np.concatenate(
    [
        np.einsum('iaL->aiL',
                  np.array([XU_aiL[:,0,...], XUm_aiL[:,0,...]])),
        np.einsum('iaL->aiL',
                  np.array([XU_aiL[:,1,...], XUm_aiL[:,1,...]]))
    ], axis=-1
)

In [25]:
fig, ax = plt.subplots(1,1)
fig.frameon = False
ax.plot(X_aiL[0],X_aiL[1],color='black', marker="o");
ax.plot(XU_aiL[0],XU_aiL[1],color='green', marker="o");
ax.plot(XUm_aiL[0], XUm_aiL[1],color='red', marker="o");
ax.plot(XUIm_aiL[0], XUIm_aiL[1],color='blue', marker="o");
ax.set_xlabel('X');
ax.set_ylabel('Y');
ax.set_aspect('equal')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [26]:
XU_Lai = np.einsum('aiL->Lai',XU_aiL)
XUm_Lai = np.einsum('aiL->Lai',XUm_aiL)
XUIm_Lai = np.einsum('aiL->Lai',XUIm_aiL)

In [27]:
fig = plt.figure()
ax = plt.axes(projection="3d")
for X_ai,XU_ai, XUm_ai in zip(X_Lai, XU_Lai, XUm_Lai):
    x_line, y_line, z_line = X_ai
    ax.plot3D(x_line, y_line, z_line, 'gray');
    x_line, y_line, z_line = XU_ai
    ax.plot3D(x_line, y_line, z_line, 'green')
    x_line, y_line, z_line = XUm_ai
    ax.plot3D(x_line, y_line, z_line, 'red')
for XUIm_ai in XUIm_Lai:
    x_line, y_line, z_line = XUIm_ai
    ax.plot3D(x_line, y_line, z_line, 'blue')    
ax.scatter3D(x_points, y_points, z_points, color='red');
ax.set_xlabel('X');
ax.set_ylabel('Y');
ax.set_zlabel('Z');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Local displacement at interface points
Transform the displacements at the interface planes $i$ to the local base of the link $L$
\begin{align}
    u^\mathrm{m}_{Lib} = T_{Lba} U^\mathrm{m}_{Lia}
\end{align}

In [28]:
um_Lib = np.einsum('...Lba,...Lia->...Lib', nT_Lba, Um_Lia)

```python
um_Lib = np.einsum('...Lba,...Lia->...Lib', nT_Lba, Um_Lia)
```

## Relative displacements at the interface $L$
The relative displacement
at the interface between the aggregates $L0$ and $L1$ along the link $L$
is given as
\begin{align}
    u^{\mathrm{m}}_{La} =  u^{\mathrm{m}}_{L1a} - u^{\mathrm{m}}_{L0a}.
\end{align}
To express it using Einstein summation rule let us introduce a sign switch operator
$(-1)^i$ and rewrite this equations a
\begin{align}
    u^{\mathrm{m}}_{La} = (-1)^i \delta_{ii} u^{\mathrm{m}}_{Lia}
\end{align}

In [29]:
switch_sign = np.array([-1,1],dtype=np.float_)
em_Lb = np.einsum('i,ii,...Lia->...La', 
                  switch_sign, DELTA2, um_Lib)
em_Lb

array([[ 0.28284271, -0.16144558,  0.        ],
       [-0.28284271, -0.16144558, -0.4712389 ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ]])

```python
switch_sign = np.array([-1,1],dtype=np.float_)
em_Lb = np.einsum('i,ii,...Lia->...La', 
                  switch_sign, DELTA2, um_Lib)
```

## Putting it all together

### Kinematic operator $B_{pLiac}$
To include all the steps describe above in a kinematic operator 
that can be evaluate prior to the non-linear calculation, let us perform
the backward substitution starting from the interface. The resulting 
multidimensional array has the form:
\begin{align}
    u^{\mathrm{m}}_{La} &= (-1)^i \delta_{ii} u^{\mathrm{m}}_{Lia} \\
    &= (-1)^i \delta_{ii} T_{Lab} U^\mathrm{m}_{Lib}  \\
    &= (-1)^i \delta_{ii} T_{Lab} (U^{\mathrm{m,trans}}_{Lib} + U^{\mathrm{m,rot}}_{Lib})\\
    &= (-1)^i \delta_{ii} T_{Lab} (U_{Lib} + \epsilon_{bcd}  \Theta_{Lic} \Delta x^{\mathrm{m}}_{Lid} ) \\
    &= S_{Liac} U_{Lic} +  S_{Liab} \epsilon_{bcd} \Delta x^{\mathrm{m}}_{Lid} \Theta_{Lic} 
\end{align}

The terns associated with the translation are summarized in 
\begin{align}
 S_{Liac} = (-1)^i \delta_{ii} T_{Lab}
\end{align}

In [66]:
S_Liac = np.einsum('i,ii,...Lac->...Liac', 
                    switch_sign, DELTA2, nT_Lba)

To provide a compact mapping, let us stack the global displacement variables $U_{Lib}$ and $\Theta_{Lic}$ along a new dimension $p$ to separate the kinematic operator from the unknown nodal variables
\begin{align}
  {u}^{\mathrm{m}}_{La} = \mathcal{B}_{pLiac} \mathcal{U}_{pLiac} 
\end{align}
where
\begin{align}
\mathcal{B}_{pLiac} =   \left[
  \begin{array}{cc}
  S_{Liac},& S_{Liab} \epsilon_{bcd} \Delta x^{\mathrm{m}}_{Lid}
  \end{array}
  \right]
\end{align}
and
\begin{align}
  \mathcal{U}_{pLiac} =   \left[
  \begin{array}{cc}
  U_{Lic},& \Theta_{Lic}
  \end{array}
  \right]
\end{align}

In [67]:
B_pLiac = np.array([S_Liac, np.einsum('Liab,bcd,Lid->Liac', S_Liac, EPS, dXm_Lia)])

In [69]:
U_pLia = np.array([U_Lia, Phi_Lia])
U_Lia

array([[[ 0. ,  0. ,  0. ],
        [ 0. , -0.1,  0. ]]])

### Verification of the compact operator

In [70]:
uu_m_La = np.einsum('pLiac,pLic->La', B_pLiac, U_pLia)

Verify the equivalence of the rearanged kinematic operator with the step by step evaluation of the response performed earlier.

In [71]:
uu_m_La - em_Lb

array([[0., 0., 0.]])

The implementation of the kinematic operator mapping the nodal displacements and rotations 
that can be cached as a property in the domain object looks as follows

### Sliding / opening inter-aggregate stiffness matrix

In [72]:
X_Lia = X_Ia[I_Li]
DELTA2 = np.identity(2)
Xm_La = np.einsum('ii,Lia->La', DELTA2, X_Lia) / 2
Xm_Lia = Xm_La[..., np.newaxis, :]
dXm_Lia = Xm_Lia - X_Lia
switch_sign = np.array([-1, 1], dtype=np.float_)
S_Liac = np.einsum('i,ii,...Lac->...Liac',
                   switch_sign, DELTA2, nT_Lba)
B_pLiac = np.array(
    [S_Liac, np.einsum('Liab,bcd,Lid->Liac',
                       S_Liac, EPS, dXm_Lia)]
)
B_Lipac = np.einsum('pLiac->Lipac', B_pLiac)

In [73]:
D_ab = np.identity(3)
K_Lipbjqd = np.einsum('Lipab,Ljqcd,ac->Lipbjqd', B_Lipac, B_Lipac, D_ab)

In [74]:
K_Lipbjqd.shape

(1, 2, 2, 3, 2, 2, 3)

Number of degrees of freedom $n_o = n_L n_i n_p n_a$ 

In [77]:
n_o = len(X_Ia) * 6 *2
n_o

24

In [78]:
K_mtx.reshape(-1,n_o,n_o)

NameError: name 'K_mtx' is not defined